In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

**Kats (Kits to Analyze Time Series)** is a light-weight, easy-to-use, extenable, and generalizable framework to perform time series analysis in Python. Time series analysis is an essential component of data science and engineering work. Kats aims to provide a one-stop shop for techniques for univariate and multivariate time series

In [ ]:
!pip install kats

In [ ]:
from kats.consts import TimeSeriesData
bitcoin=pd.read_csv('../input/bitcoin/BTC-USD.csv')
bitcoin.head()

In [ ]:
# Sort the data based on Date
bitcoin = bitcoin.sort_values(by = ['Date'])
bitcoin.head()

In [ ]:
# missing value 
bitcoin[pd.isnull(bitcoin).any(axis=1)]

In [ ]:
bitcoin=bitcoin.dropna()
bitcoin[pd.isnull(bitcoin).any(axis=1)]

In [ ]:
def trading_window(data):
    n=1
    data['Target'] = data[['Close']].shift(-n)
    return data

In [ ]:
bitcoin=bitcoin[['Date','Close','Volume']]
bitcoin.head()

In [ ]:
bitcoin['Date']=bitcoin['Date'].apply(pd.to_datetime)
bitcoin.head()

In [ ]:
bitcoin = bitcoin.rename(columns = {"Date":"time"})
bitcoin.head()

In [ ]:
bitcoin=trading_window(bitcoin)
bitcoin.head()

In [ ]:
bitcoin=bitcoin[:-1]
bitcoin.head()

In [ ]:
target=bitcoin[['time','Target']]
target.head()

In [ ]:
target_ts = TimeSeriesData(target)
print(type(target_ts))

In [ ]:
%matplotlib inline

# Must pass the name of the value columns to plot
target_ts.plot(cols=['Target'])
plt.grid()


In [ ]:
target_ts.to_dataframe().head()

In [ ]:
# import the param and model classes for Prophet model
from kats.models.prophet import ProphetModel, ProphetParams

# create a model param instance
params = ProphetParams(seasonality_mode='multiplicative') # additive mode gives worse results

# create a prophet model instance
m = ProphetModel(target_ts, params)

# fit model simply by calling m.fit()
m.fit()

# make prediction for next 360 days
fcst = m.predict(steps=360, freq="D")

In [ ]:
m.plot()

In [ ]:
fcst.head()

In [ ]:
fcst.plot(x='time',y=['fcst','fcst_lower','fcst_upper'],figsize=(20,5))
plt.legend()
plt.grid()

# Feature Extraction with Kats

In [ ]:
# Initiate feature extraction class
from kats.tsfeatures.tsfeatures import TsFeatures
tsFeatures = TsFeatures()

In [ ]:
features_target = TsFeatures().transform(target_ts)
features_target